In [1]:
score_code = '''
import numpy as np
from sklearn.base import BaseEstimator
from typing import Tuple

def score(text: str, model: BaseEstimator, threshold: float) -> Tuple[bool, float]:
    if hasattr(model, "predict_proba"):
        prob = model.predict_proba([text])[0][1]
    elif hasattr(model, "decision_function"):
        prob = model.decision_function([text])[0]
        prob = 1 / (1 + np.exp(-prob))
    else:
        raise ValueError("Model must implement predict_proba or decision_function.")
    
    prediction = prob >= threshold
    return prediction, float(prob)
'''

with open("score.py", "w") as f:
    f.write(score_code.strip())


In [2]:
app_code = '''
from flask import Flask, request, jsonify
import joblib
from score import score
from sklearn.linear_model import LogisticRegression

def create_app():
    app = Flask(__name__)
    model: LogisticRegression = joblib.load("best_model.pkl")

    @app.route("/score", methods=["POST"])
    def score_endpoint():
        data = request.json
        text = data.get("text", "")
        threshold = float(data.get("threshold", 0.5))
        
        prediction, propensity = score(text, model, threshold)
        return jsonify({"prediction": int(prediction), "propensity": propensity})

    return app

if __name__ == "__main__":
    app = create_app()
    app.run(host="0.0.0.0", port=5000)
'''

with open("app.py", "w") as f:
    f.write(app_code.strip())


In [3]:
test_code = '''
import numpy as np
from score import score

def test_score():
    """Unit test for the score function."""
    from sklearn.linear_model import LogisticRegression

    model = LogisticRegression()
    model.classes_ = np.array([0, 1])
    model.predict_proba = lambda x: np.array([[0.3, 0.7]])

    text = "This is a spam message!"
    threshold = 0.5

    prediction, propensity = score(text, model, threshold)

    assert isinstance(prediction, (bool, np.bool_)), "Prediction should be a boolean."
    assert isinstance(propensity, float), "Propensity should be a float."
    assert 0 <= propensity <= 1, "Propensity score should be between 0 and 1."
    assert score(text, model, 0)[0] == True, "With threshold 0, prediction should be True."
    assert score(text, model, 1)[0] == False, "With threshold 1, prediction should be False."
'''

with open("test.py", "w") as f:
    f.write(test_code.strip())


In [4]:
docker_code = '''
FROM python:3.10-slim

WORKDIR /app

COPY app.py score.py best_model.pkl requirements.txt ./

RUN pip install --no-cache-dir -r requirements.txt

CMD ["python", "app.py"]
'''

with open("Dockerfile", "w") as f:
    f.write(docker_code.strip())


In [5]:
reqs = '''
flask
scikit-learn
joblib
'''

with open("requirements.txt", "w") as f:
    f.write(reqs.strip())


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
import joblib

# Sample data
X = ["win money now", "click here", "hello friend", "let's meet", "free lottery", "how are you"]
y = [1, 1, 0, 0, 1, 0]  # 1 = spam, 0 = not spam

# Build pipeline: vectorizer + model
pipe = make_pipeline(CountVectorizer(), LogisticRegression())
pipe.fit(X, y)

# Save to disk
joblib.dump(pipe, "best_model.pkl")


['best_model.pkl']

In [7]:
import os
import subprocess
import time
import requests

def test_docker():
    """Integration test for the Docker container running the Flask app."""
    
    # Step 1: Build the Docker image
    build_cmd = "docker build -t flask-score-app ."
    build_status = subprocess.run(build_cmd, shell=True)
    assert build_status.returncode == 0, "Docker image build failed."
    
    # Step 2: Run the Docker container
    run_cmd = "docker run -d -p 5000:5000 --name score_container flask-score-app"
    container_id = subprocess.check_output(run_cmd, shell=True).decode().strip()
    
    # Step 3: Wait for the container to be ready
    time.sleep(5)
    
    # Step 4: Send request to the /score endpoint
    url = "http://127.0.0.1:5000/score"
    data = {"text": "This is spam!", "threshold": 0.5}
    response = requests.post(url, json=data)
    
    # Step 5: Check response
    assert response.status_code == 200, "Expected 200 OK from Flask app."
    json_response = response.json()
    assert "prediction" in json_response and "propensity" in json_response, "Response JSON is missing fields."
    
    print("Test passed ✅ Response:", json_response)
    
    # Step 6: Stop and remove the container
    subprocess.run("docker stop score_container", shell=True)
    subprocess.run("docker rm score_container", shell=True)


In [8]:
!pip install pytest pytest-cov


In [9]:
!pytest test.py --cov=. --cov-report=term > coverage.txt
!cat coverage.txt



============================= test session starts ==============================
platform darwin -- Python 3.11.7, pytest-7.4.0, pluggy-1.0.0
rootdir: /Users/suryaeada
plugins: anyio-4.2.0, hydra-core-1.3.2, cov-6.1.1
collected 1 item

test.py .                                                                [100%]

================================ tests coverage ================================
_______________ coverage: platform darwin, python 3.11.7-final-0 _______________

Name       Stmts   Miss  Cover
------------------------------
app.py        18     18     0%
score.py      12      4    67%
test.py       15      0   100%
------------------------------
TOTAL         45     22    51%
============================== 1 passed in 0.59s ===============================
